In [1]:
import pandas as pd

df = pd.read_csv("../data/geocoded_data/merged_2021_2025.csv")

# Parse datetime columns
datetime_cols = ["Opened", "Closed Date 1", "Closed Date 2", "closed"]
for col in datetime_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# Ensure numeric types
df["resolution_time_hours"] = pd.to_numeric(df["resolution_time_hours"], errors="coerce")
df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
df["lng"] = pd.to_numeric(df["lng"], errors="coerce")

# Standardize boolean columns
bool_cols = ["has_closure_time", "ok"]
for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.lower().map({"true": True, "false": False})


C:\Users\shrey\AppData\Local\Temp\ipykernel_28848\1891512058.py:3: DtypeWarning: Columns (1,3,4,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/geocoded_data/merged_2021_2025.csv")


In [2]:
#Cleanup of descriptions:
print("Length of df before cleanup:",len(df))
df = df[df["Description"].notna()]
df = df[df["Description"].str.strip() != ""]
df = df.drop_duplicates(subset=["Description", "Address", "Opened"])
print("Length of df after cleanup:", len(df))

Length of df before cleanup: 443308
Length of df after cleanup: 273825


In [3]:
#Fetch unique data for scalable workload

df["Description_norm"] = (
    df["Description"]
    .str.strip()
    .str.lower()
)

unique_desc_df = df[["Description_norm"]].drop_duplicates().reset_index(drop=True)

print("Original rows:", len(df))
print("Unique descriptions:", len(unique_desc_df))




Original rows: 273825
Unique descriptions: 909


In [4]:
import re

def normalize(text):
    text = text.lower().strip()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

unique_desc_df["clean_text"] = unique_desc_df["Description_norm"].apply(normalize)


In [6]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
#Categorize them!
#Method 1

tfidf = TfidfVectorizer(
    max_features=2000,
    ngram_range=(1,2),
    stop_words="english"
)

X_tfidf = tfidf.fit_transform(unique_desc_df["clean_text"])

k = 10
kmeans_tfidf = KMeans(n_clusters=k, random_state=42)
unique_desc_df["cluster_tfidf"] = kmeans_tfidf.fit_predict(X_tfidf)

sil_tfidf = silhouette_score(X_tfidf, unique_desc_df["cluster_tfidf"])
print("TF-IDF Silhouette:", sil_tfidf)



TF-IDF Silhouette: 0.017114081304171375


In [8]:
#Method 2

from sentence_transformers import SentenceTransformer


model = SentenceTransformer("all-MiniLM-L6-v2")
X_embed = model.encode(unique_desc_df["clean_text"].tolist(), show_progress_bar=True)
kmeans_embed = KMeans(n_clusters=k, random_state=42)
unique_desc_df["cluster_embed"] = kmeans_embed.fit_predict(X_embed)

sil_embed = silhouette_score(X_embed, unique_desc_df["cluster_embed"])
print("Embedding Silhouette:", sil_embed)


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 397.51it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 29/29 [00:02<00:00, 13.63it/s]
c:\Users\shrey\anaconda3\envs\geo\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


Embedding Silhouette: 0.04657798260450363


In [9]:
#Method 3
from sklearn.cluster import AgglomerativeClustering

agg = AgglomerativeClustering(n_clusters=k)
unique_desc_df["cluster_hier"] = agg.fit_predict(X_embed)

sil_hier = silhouette_score(X_embed, unique_desc_df["cluster_hier"])
print("Hierarchical Silhouette:", sil_hier)


Hierarchical Silhouette: 0.03747142106294632


In [10]:
desc_counts = df["Description"].value_counts()

print(desc_counts.head(20))
print("\nTop 50 descriptions cover:",
      desc_counts.head(50).sum() / len(df))


Description
Bulk Collection Request                                                                    39070
Missed Residential Garbage                                                                 12900
Damaged Garbage Cart Replacement                                                           11949
Missing Garbage Cart                                                                        8513
Missed Yard Trimmings                                                                       7602
Damaged Recycling Cart Replacement                                                          7282
Missing Recycling Cart                                                                      6329
Account Information For Active Water And Sewer Account                                      5707
Business License Renewal-City of Atlanta                                                    5156
Property Owner or Renter Request to Establish New Water and Sewer Bill Account Services     5022
Code Enforcement -

In [11]:
categories = [
    "Waste Management",
    "Water & Sewer",
    "Road & Infrastructure",
    "Code Enforcement",
    "Licensing & Permits",
    "Administrative / Account Services",
    "Public Safety",
    "Other"
]


In [12]:
category_anchors = {
    "Waste Management": "garbage recycling bulk trash cart yard trimmings waste pickup",
    "Water & Sewer": "water sewer leak meter no water account bill adjustment",
    "Road & Infrastructure": "pothole street repair sidewalk road damage",
    "Code Enforcement": "junk debris violation private property complaint",
    "Licensing & Permits": "business license permit renewal application",
    "Administrative / Account Services": "account information transfer close billing inquiry",
    "Public Safety": "hazard dangerous emergency complaint",
}


In [13]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

# Embed unique descriptions
desc_embeddings = model.encode(
    unique_desc_df["Description_norm"].tolist(),
    show_progress_bar=True
)

# Embed anchor texts
anchor_embeddings = model.encode(list(category_anchors.values()))


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 344.44it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 29/29 [00:02<00:00, 13.54it/s]


In [14]:
similarity_matrix = cosine_similarity(desc_embeddings, anchor_embeddings)

# Get best matching category index
best_match_idx = np.argmax(similarity_matrix, axis=1)

# Map index back to category name
category_names = list(category_anchors.keys())
unique_desc_df["category"] = [category_names[i] for i in best_match_idx]


In [15]:
unique_desc_df.to_csv("../data/geocoded_data/categories_new.csv", index=False)


In [16]:
### Map it back to the original data
import pandas as pd

unique_desc_df = pd.read_csv("../data/geocoded_data/categories_new.csv")


In [17]:

unique_desc_df["category"].value_counts()


category
Public Safety                        179
Code Enforcement                     176
Waste Management                     138
Water & Sewer                        137
Licensing & Permits                  124
Road & Infrastructure                100
Administrative / Account Services     55
Name: count, dtype: int64

In [18]:
# Map back to full dataset

desc_to_cat = dict(
    zip(unique_desc_df["Description_norm"], unique_desc_df["category"])
)

df["category"] = df["Description_norm"].map(desc_to_cat)


In [19]:
df.columns

Index(['Opened', 'Description', 'Address', 'Zip Code', 'Closed Date 1',
       'Closed Date 2', 'Status', 'Number', 'closed', 'resolution_time_hours',
       'has_closure_time', 'zip_clean', 'zip_int', 'address_norm', 'lat',
       'lng', 'ok', 'year', 'Short Description', 'Description_norm',
       'category'],
      dtype='object')

In [20]:
df["category"].value_counts()


category
Waste Management                     127321
Water & Sewer                         49926
Code Enforcement                      43899
Road & Infrastructure                 17397
Public Safety                         13276
Administrative / Account Services     11605
Licensing & Permits                   10401
Name: count, dtype: int64

In [30]:
for _, row in unique_desc_df.iterrows():
    print(f"{row['Description_norm']}  -->  {row['category']}")


information on how to request an account adjustment  -->  Administrative / Account Services
information on how to dispute your water and sewer bill  -->  Water & Sewer
street light bulb replacement or street light is out - missing  -->  Road & Infrastructure
right of way maintenance visibility/overgrowth issue  -->  Road & Infrastructure
property owner or renter request to establish new water and sewer bill account services  -->  Water & Sewer
mail/fax request on water/sewer accounts  -->  Water & Sewer
code enforcement - junk, trash, and debris-private property only  -->  Code Enforcement
misapplied payments on a water and sewer account  -->  Water & Sewer
deposit or adjustment refund request on a water and sewer account  -->  Water & Sewer
code enforcement general information  -->  Code Enforcement
missed residential recycling backyard service  -->  Waste Management
customer reports a leaking or damaged fire hydrant  -->  Public Safety
bulk collection request  -->  Waste Management
s

In [21]:
df.to_csv("../data/geocoded_data/merged_data_with_categories2.csv", index=False)

In [22]:
category_scores = {
    "Public Safety": 10,                       # emergency, danger
    "Water & Sewer": 9,                        # flooding, sanitation risk
    "Road & Infrastructure": 8,                # traffic, physical risk
    "Waste Management": 7,                     # hygiene impact
    "Code Enforcement": 6,                     # compliance issues
    "Licensing & Permits": 5,                  # business impact
    "Administrative / Account Services": 4,    # billing/info
    "Other": 3                                 # uncategorized
}

In [23]:
df["category_score"] = df["category"].map(category_scores)

In [24]:
print(df["category_score"].isna().sum())

0


In [25]:
max_score = max(category_scores.values())

df["category_score_norm"] = df["category_score"] / max_score

In [26]:
df.to_csv("../data/geocoded_data/merged_data_with_categories_score.csv", index=False)

In [27]:
df.columns

Index(['Opened', 'Description', 'Address', 'Zip Code', 'Closed Date 1',
       'Closed Date 2', 'Status', 'Number', 'closed', 'resolution_time_hours',
       'has_closure_time', 'zip_clean', 'zip_int', 'address_norm', 'lat',
       'lng', 'ok', 'year', 'Short Description', 'Description_norm',
       'category', 'category_score', 'category_score_norm'],
      dtype='object')